In [13]:
import pandas as pd
import numpy as np

In [4]:
# Load datasets
# lap_times = pd.read_csv("lap_times.csv")
# pit_stops = pd.read_csv("pit_stops.csv")
qualifying = pd.read_csv("data/qualifying.csv")
qualifying = qualifying.drop(columns=['qualifyId'])
races = pd.read_csv("data/races.csv")
races = races.drop(columns=['url'])
results = pd.read_csv("data/results.csv")
results = results.drop(columns=['resultId'])
# seasons = pd.read_csv("seasons.csv")
# sprint_results = pd.read_csv("sprint_results.csv")
# status = pd.read_csv("status.csv")
circuits = pd.read_csv("data/circuits.csv")
circuits = circuits.drop(columns=['url'])
constructor_results = pd.read_csv("data/constructor_results.csv")
constructor_standings = pd.read_csv("data/constructor_standings.csv")
constructors = pd.read_csv("data/constructors.csv")
constructors = constructors.drop(columns=['url'])
# driver_standings = pd.read_csv("driver_standings.csv")
drivers = pd.read_csv("data/drivers.csv")
drivers = drivers.drop(columns=['url'])

In [5]:
def label_encoder(col):
    labelEncoder = LabelEncoder()
    encoded_col = labelEncoder.fit_transform(col)
    return encoded_col

In [6]:
# Merging essential datasets
merged_data = races.merge(results, on="raceId", how="left", suffixes=("", "_results"))
merged_data = merged_data.merge(drivers, on="driverId", how="left", suffixes=("", "_drivers"))
merged_data = merged_data.merge(constructors, on="constructorId", how="left", suffixes=("", "_constructors"))

# Merging potentially useful datasets
merged_data = merged_data.merge(circuits, on="circuitId", how="left", suffixes=("", "_circuits"))
merged_data = merged_data.merge(qualifying, on=["raceId", "driverId"], how="left", suffixes=("", "_qualifying"))
merged_data = merged_data.merge(constructor_results, on=["raceId", "constructorId"], how="left", suffixes=("", "_constructorResults"))
merged_data = merged_data.merge(constructor_standings, on="constructorId", how="left", suffixes=("", "_constructorStandings"))

# Extracting date components
merged_data['date'] = pd.to_datetime(merged_data['date'])
merged_data['month'] = merged_data['date'].dt.month
merged_data['day'] = merged_data['date'].dt.day
merged_data['day_of_week'] = merged_data['date'].dt.dayofweek

# Cyclical encoding - encoding cyclical features using sine and cosine transformations
merged_data['month_sin'] = np.sin(2 * np.pi * merged_data['month'] / 12)
merged_data['month_cos'] = np.cos(2 * np.pi * merged_data['month'] / 12)
merged_data['day_sin'] = np.sin(2 * np.pi * merged_data['day'] / 31)
merged_data['day_cos'] = np.cos(2 * np.pi * merged_data['day'] / 31)
merged_data['day_of_week_sin'] = np.sin(2 * np.pi * merged_data['day_of_week'] / 7)

# Label Encoder for Categorical features
merged_data['name'] = label_encoder(merged_data['name'])
merged_data['nationality'] = label_encoder(merged_data['nationality'])
merged_data['driverRef'] = label_encoder(merged_data['driverRef'])
merged_data['circuitRef'] = label_encoder(merged_data['circuitRef'])
merged_data['constructorRef'] = label_encoder(merged_data['constructorRef'])
merged_data['name_circuits'] = label_encoder(merged_data['name_circuits'])

In [7]:
merged_data.columns

Index(['raceId', 'year', 'round', 'circuitId', 'name', 'date', 'time',
       'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time',
       'quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'driverId',
       'constructorId', 'number', 'grid', 'position', 'positionText',
       'positionOrder', 'points', 'laps', 'time_results', 'milliseconds',
       'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed', 'statusId',
       'driverRef', 'number_drivers', 'code', 'forename', 'surname', 'dob',
       'nationality', 'constructorRef', 'name_constructors',
       'nationality_constructors', 'circuitRef', 'name_circuits', 'location',
       'country', 'lat', 'lng', 'alt', 'constructorId_qualifying',
       'number_qualifying', 'position_qualifying', 'q1', 'q2', 'q3',
       'constructorResultsId', 'points_constructorResults', 'status',
       'constructorStandingsId', 'raceId_constructorStandings',
       'points_constructorStandings', 'position_constructorStandin

In [8]:
unuseful_cols = ['raceId', 'circuitId', 'date', 'time', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time',
       'quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'driverId', 'constructorId', 'number', 'position', 'positionText', 'time_results',
       'fastestLapTime', 'statusId', 'code', 'forename', 'surname', 'dob', 'name_constructors', 'nationality_constructors', 'location', 'country',
       'constructorId_qualifying', 'q1', 'q2', 'q3', 'constructorResultsId', 'status', 'constructorStandingsId', 'raceId_constructorStandings',
       'positionText_constructorStandings', 'month', 'day', 'day_of_week']
merged_data = merged_data.drop(columns=unuseful_cols)

# Fill NaNs for specific columns
merged_data['grid'].fillna(merged_data['grid'].max() + 1, inplace=True)
merged_data['laps'].fillna(0, inplace=True)
merged_data['positionOrder'] = pd.to_numeric(merged_data['positionOrder'], errors='coerce')
merged_data['positionOrder'].fillna(merged_data['positionOrder'].max() + 1, inplace=True)
merged_data['positionOrder'] = merged_data['positionOrder'].astype('int64')

timing_cols = ['milliseconds', 'fastestLapSpeed', 'position_constructorStandings']
for col in timing_cols:
    merged_data[col] = pd.to_numeric(merged_data[col], errors='coerce')
    merged_data[col].fillna(merged_data[col].max() + 1, inplace=True)

# For 'points' and 'wins', NaN means the driver didn't earn any points or wins, hence filling NaN with 0
merged_data['points'].fillna(0, inplace=True)
merged_data['wins'].fillna(0, inplace=True)

# For 'fastestLap', 'rank', and 'fastestLapTime', as they relate to fastest lap metrics, if its NaN, it might mean the data was not recorded, so filling them with some identifier values.
merged_data['fastestLap'].fillna(-1, inplace=True)
merged_data['fastestLap'] = np.where(merged_data['fastestLap'] == '\\N', -1, merged_data['fastestLap'])
merged_data['rank'] = pd.to_numeric(merged_data['rank'], errors='coerce')
merged_data['rank'].fillna(merged_data['rank'].max() + 1, inplace=True)
merged_data['rank'] = np.where(merged_data['rank'] == '\\N', merged_data['rank'].max() + 1, merged_data['rank'])

# For 'number_drivers' and 'number_qualifying', 'position_qualifying',  filling NaN with '-1' identifiers
merged_data['number_drivers'].fillna(-1, inplace=True)
merged_data['number_drivers'] = np.where(merged_data['number_drivers'] == '\\N', -1, merged_data['number_drivers'])
merged_data['number_qualifying'].fillna(-1, inplace=True)
merged_data['position_qualifying'].fillna(-1, inplace=True)

# For 'points_constructorResults' and 'points_constructorStandings', if NaN, it might mean no points were earned, so filling them with 0
merged_data['points_constructorResults'].fillna(0, inplace=True)
merged_data['points_constructorStandings'].fillna(0, inplace=True)

merged_data['alt'] = np.where(merged_data['alt'] == '\\N', 0, merged_data['alt'])
cols_to_float = merged_data.columns.difference(['year', 'round', 'positionOrder'])
merged_data[cols_to_float] = merged_data[cols_to_float].astype('float32')

In [9]:
merged_data.columns

Index(['year', 'round', 'name', 'grid', 'positionOrder', 'points', 'laps',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapSpeed', 'driverRef',
       'number_drivers', 'nationality', 'constructorRef', 'circuitRef',
       'name_circuits', 'lat', 'lng', 'alt', 'number_qualifying',
       'position_qualifying', 'points_constructorResults',
       'points_constructorStandings', 'position_constructorStandings', 'wins',
       'month_sin', 'month_cos', 'day_sin', 'day_cos', 'day_of_week_sin'],
      dtype='object')

In [10]:
merged_data

,year,round,name,grid,positionOrder,points,laps,milliseconds,fastestLap,rank,...,position_qualifying,points_constructorResults,points_constructorStandings,position_constructorStandings,wins,month_sin,month_cos,day_sin,day_cos,day_of_week_sin
0,2009,1,3.0,1.0,1,10.0,58.0,5655784.0,17.0,3.0,...,1.0,18.0,18.0,1.0,1.0,1.000000,6.123234e-17,-0.394356,0.918958,-0.781832
1,2009,1,3.0,1.0,1,10.0,58.0,5655784.0,17.0,3.0,...,1.0,18.0,25.0,1.0,2.0,1.000000,6.123234e-17,-0.394356,0.918958,-0.781832
2,2009,1,3.0,1.0,1,10.0,58.0,5655784.0,17.0,3.0,...,1.0,18.0,36.0,1.0,2.0,1.000000,6.123234e-17,-0.394356,0.918958,-0.781832
3,2009,1,3.0,1.0,1,10.0,58.0,5655784.0,17.0,3.0,...,1.0,18.0,50.0,1.0,3.0,1.000000,6.123234e-17,-0.394356,0.918958,-0.781832
4,2009,1,3.0,1.0,1,10.0,58.0,5655784.0,17.0,3.0,...,1.0,18.0,68.0,1.0,4.0,1.000000,6.123234e-17,-0.394356,0.918958,-0.781832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9523628,2023,18,52.0,35.0,40,0.0,0.0,15090541.0,-1.0,25.0,...,-1.0,0.0,0.0,23.0,0.0,-0.866025,5.000000e-01,-0.968077,-0.250653,-0.781832
9523629,2023,19,31.0,35.0,40,0.0,0.0,15090541.0,-1.0,25.0,...,-1.0,0.0,0.0,23.0,0.0,-0.866025,5.000000e-01,-0.394356,0.918958,-0.781832
9523630,2023,20,49.0,35.0,40,0.0,0.0,15090541.0,-1.0,25.0,...,-1.0,0.0,0.0,23.0,0.0,-0.500000,8.660254e-01,0.848644,0.528964,-0.781832
9523631,2023,21,27.0,35.0,40,0.0,0.0,15090541.0,-1.0,25.0,...,-1.0,0.0,0.0,23.0,0.0,-0.500000,8.660254e-01,-0.651372,-0.758758,-0.781832


In [11]:
merged_data.dtypes

year                               int64
round                              int64
name                             float32
grid                             float32
positionOrder                      int64
points                           float32
laps                             float32
milliseconds                     float32
fastestLap                       float32
rank                             float32
fastestLapSpeed                  float32
driverRef                        float32
number_drivers                   float32
nationality                      float32
constructorRef                   float32
circuitRef                       float32
name_circuits                    float32
lat                              float32
lng                              float32
alt                              float32
number_qualifying                float32
position_qualifying              float32
points_constructorResults        float32
points_constructorStandings      float32
position_constru

In [12]:
merged_data.to_csv('data/merged_data.csv')